<a href="https://colab.research.google.com/github/roshan-adusumilli/RNN-investing-advice-generation/blob/master/investing_advice_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make the necessary imports

In [0]:
import tensorflow as tf 
import numpy as np 
import os 

Download our data, which is a .txt file of Benjamin Graham's classic investing book *Intelligent Investor*. I removed the preface, index and a few graphs from the file to help our model generate more relevant text. Once we download the file, we take a look at how many total characters are in it.

In [0]:
from google.colab import files
files.upload()

text = open('The_Intelligent_Investor copy.txt', 'rb').read().decode(encoding='utf-8')

print ('Length of text: {} characters'.format(len(text)))


Saving The_Intelligent_Investor copy.txt to The_Intelligent_Investor copy (2).txt
Length of text: 1327032 characters


We'll see how many unique characters exist in the file as well as map our characters to integers to later train our model.

In [0]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

#Maps characters to ints 
char2int = {char: num for num, char in enumerate(vocab)}
#Maps ints to characters 
int2char = np.array(vocab)
#Intelligent Investor text represented as ints.
text_as_int = np.array([char2int[char] for char in text])

print('\n')
print(char2int)
print('\n')
print(int2char)

110 unique characters


{'\x01': 0, '\x02': 1, '\x03': 2, '\n': 3, '\x0c': 4, '\r': 5, ' ': 6, '!': 7, '"': 8, '#': 9, '$': 10, '%': 11, '&': 12, '(': 13, ')': 14, '*': 15, '+': 16, ',': 17, '-': 18, '.': 19, '/': 20, '0': 21, '1': 22, '2': 23, '3': 24, '4': 25, '5': 26, '6': 27, '7': 28, '8': 29, '9': 30, ':': 31, ';': 32, '=': 33, '?': 34, 'A': 35, 'B': 36, 'C': 37, 'D': 38, 'E': 39, 'F': 40, 'G': 41, 'H': 42, 'I': 43, 'J': 44, 'K': 45, 'L': 46, 'M': 47, 'N': 48, 'O': 49, 'P': 50, 'Q': 51, 'R': 52, 'S': 53, 'T': 54, 'U': 55, 'V': 56, 'W': 57, 'X': 58, 'Y': 59, 'Z': 60, '[': 61, ']': 62, '_': 63, 'a': 64, 'b': 65, 'c': 66, 'd': 67, 'e': 68, 'f': 69, 'g': 70, 'h': 71, 'i': 72, 'j': 73, 'k': 74, 'l': 75, 'm': 76, 'n': 77, 'o': 78, 'p': 79, 'q': 80, 'r': 81, 's': 82, 't': 83, 'u': 84, 'v': 85, 'w': 86, 'x': 87, 'y': 88, 'z': 89, '~': 90, '¢': 91, '£': 92, 'ç': 93, 'é': 94, 'ê': 95, 'ë': 96, 'î': 97, 'ï': 98, 'ø': 99, '–': 100, '—': 101, '‘': 102, '’': 103, '“': 104, '”': 105, '†': 106, '

Our goal for the RNN model is to predict the most likely character after a given sequence of characters. To do this we will break input sequences from the text into an example sequence and target sequence.

In [0]:
#targets are the example sequences, but shifted one character to the right
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create examples and targets sequences 
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_seq(chunk):
    example_text = chunk[:-1]
    target_text = chunk[1:]
    return example_text, target_text

dataset = sequences.map(split_input_seq)

#look at the first example and target sequence
for example_text, target_text in  dataset.take(1):
  print ('Example data: ', repr(''.join(int2char[example_text.numpy()])))
  print ('Target data:', repr(''.join(int2char[target_text.numpy()])))

Example data:  '\x0c\r\n\x0c                     A Note About Benjamin Graham                        xii\r\n\r\n  How did Graham'
Target data: '\r\n\x0c                     A Note About Benjamin Graham                        xii\r\n\r\n  How did Graham '


Our current data isn't the most effective for the model, so we segment the data into batches and then shuffle it.

In [0]:
batch_size = 64

buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

Now we'll build the actual model

In [0]:
model = tf.keras.Sequential()
#add input layer
model.add(tf.keras.layers.Embedding(len(vocab), 256, batch_input_shape=[batch_size, None])) 
#add RNN layer
model.add(tf.keras.layers.GRU(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
#add output layer
model.add(tf.keras.layers.Dense(len(vocab)))

#summary of our model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           27904     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 109)           111725    
Total params: 4,077,933
Trainable params: 4,077,933
Non-trainable params: 0
_________________________________________________________________


Compile the model

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

Now we save checkpoints and train our model

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

#Make sure the weights are saved
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(dataset, epochs=30, callbacks=[checkpoint_callback])

Epoch 1/30
141/141 [==============================] - 6s 42ms/step - loss: 2.9734
Epoch 2/30
141/141 [==============================] - 6s 42ms/step - loss: 2.0764
Epoch 3/30
141/141 [==============================] - 6s 42ms/step - loss: 1.7660
Epoch 4/30
141/141 [==============================] - 6s 42ms/step - loss: 1.5416
Epoch 5/30
141/141 [==============================] - 6s 42ms/step - loss: 1.4099
Epoch 6/30
141/141 [==============================] - 6s 42ms/step - loss: 1.3258
Epoch 7/30
141/141 [==============================] - 6s 42ms/step - loss: 1.2652
Epoch 8/30
141/141 [==============================] - 6s 42ms/step - loss: 1.2168
Epoch 9/30
141/141 [==============================] - 6s 42ms/step - loss: 1.1745
Epoch 10/30
141/141 [==============================] - 6s 42ms/step - loss: 1.1367
Epoch 11/30
141/141 [==============================] - 6s 42ms/step - loss: 1.1013
Epoch 12/30
141/141 [==============================] - 6s 42ms/step - loss: 1.0662
Epoch 13/30
1

RNNs only accept a fixed batch size and we want to change the batch size to 1 to make the prediction simpler, so we will need to rebuild the model. We previously saved the weights for our batch size of 64, so we can restore them now


In [0]:
model = tf.keras.Sequential()
#add input layer
model.add(tf.keras.layers.Embedding(len(vocab), 256, batch_input_shape=[1, None])) 
#add RNN layer
model.add(tf.keras.layers.GRU(1024, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
#add output layer
model.add(tf.keras.layers.Dense(len(vocab)))
#load weights from previous model
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

#summary of our model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            27904     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 109)            111725    
Total params: 4,077,933
Trainable params: 4,077,933
Non-trainable params: 0
_________________________________________________________________


Finally we get to the part we are waiting for: the text generation!

In [0]:
#lower temperature gives more predicatable text, higher temperature gives more surprising text
#try any temperature in the range of 0.1 to 1
def generate_text(model, start_string, temperature):

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2int[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(int2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string="Advice: ", temperature=.5))

Advice: “In its businesses and more on “ashight”? You can use “portfolio trackers” at websites like www.morningstar.com or conservative
companies, including Barrard & Poor’s—are investing on the other hand, if
his company’s earnings as if it were the best to use the rest of your tax
swings down to the conservative investor in companies that are seriously only a minimum
of 40% fully offsets a predection of a company’s capital in a separate company. The first is the perfect record
of high-grade bonds and preferred stocks to companies that are really disclosed at the time of convertible
bonds, and safety of price discounts and residential properties.1
    What about Exodus the business? Graham wouldn’t have to be properly bear-
ing about the future. If the analyst was above are not actively sold in 1970, the
advantage of the best professional analyst was able to then be spared the market as a
whole has also varied from one date to another. Figures
on this point for lower coupons. The “
